# <center>Feature engineering : étude de cas</center>
<center>Prédiction de prix de voitures</center>

*Objectifs* : 

Ce TP a pour but de mettre en oeuvre les méthodes relatives au feature engineering présentées en cours afin d'améliorer la prédiction du prix d'une voiture. Plus particulièrement :
- Identifier le type de chaque variable (quantitative/ordinale/catégorielle) et les transformer de façon adéquate (one hot encoding, standardisation, ...).
- Etudier les relations entre variables pour ajouter de nouvelles features.
- Comprendre la nature des features pour en ajouter de nouvelles.
- Mettre en évidence la pertinence de ces manipulations.

Dans cette séance, les librairies suivantes seront utilisées : installez-les si nécessaire et chargez-les

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as plt
import time

## Table des matières

<p><div class="lev1"><a href="#1-Jeu-de-données-:-présentation-et-analyse"><span class="toc-item-num">1 - </span>Jeu de données : présentation et analyse</a></div>
<p><div class="lev1"><a href="#2-Transformations-essentielles"><span class="toc-item-num">2 - </span>Transformations essentielles</a></div>
<p><div class="lev2"><a href="#2-1-One-hot-encoding"><span class="toc-item-num">2-1 </span>One-hot encoding</a></div>
<p><div class="lev2"><a href="#2-2-Mise-à-l'échelle"><span class="toc-item-num"> 2-2 </span>Mise à l'échelle</a></div>
<p><div class="lev2"><a href="#2-3-Résultat-de-référence"><span class="toc-item-num"> 2-3 </span>Résultat de référence</a></div>
<p><div class="lev2"><a href="#2-4-One-hot-encoding-vs-label-encoding"><span class="toc-item-num"> 2-4 </span>One-hot encoding vs label encoding</a></div>
<p><div class="lev2"><a href="#2-5-Oubli-de-mise-à-l'échelle"><span class="toc-item-num"> 2-5 </span>Oubli de mise à l'échelle</a></div>
<p><div class="lev2"><a href="#2-6-Sur-les-données-non-standardisées-et-mal-encodées"><span class="toc-item-num"> 2-6 </span>Sur les données non standardisées et mal encodées</a></div>
<p><div class="lev1"><a href="#3-Création-de-features"><span class="toc-item-num">3 </span>Création de features</a></div>
<p><div class="lev2"><a href="#3-1-Le-cas-de-la-variable-'Year'"><span class="toc-item-num"> 3-1 </span>Le cas de la variable 'Year'</a></div>
<p><div class="lev2"><a href="#3-2-Le-cas-de-la-variable-'highway-MPG'"><span class="toc-item-num"> 3-2 </span>Le cas de la variable 'highway MPG'</a></div>
<p><div class="lev2"><a href="#3-3-Terme-d'interaction"><span class="toc-item-num"> 3-3 </span>Terme d'interaction</a></div>
<p><div class="lev2"><a href="#3-4-Encore-des-transformations-..."><span class="toc-item-num"> 3-4 </span>Encore des transformations ...</a></div>
<p><div class="lev1"><a href="#4-Et-les-variables-catégorielles-?"><span class="toc-item-num"> 4 </span>Et les variables catégorielles ?</a></div>
<p><div class="lev2"><a href="#4-1-Sont-elles-toutes-déterminantes-?"><span class="toc-item-num"> 4-1 </span>Sont-elles toutes déterminantes ?</a></div>
<p><div class="lev1"><a href="#5-OneHotEncoder-vs-get_dummies"><span class="toc-item-num"> 5 </span>OneHotEncoder vs get_dummies</a></div>

## 1 Jeu de données : présentation et analyse

Cette séance est inspirée de [ce travail](https://www.kaggle.com/anirbank/usedcarpriceprediction) et de [celui-ci](https://www.kaggle.com/bryanb/simple-and-quick-eda) et étudie le jeu de données disponible sur [Kaggle](https://www.kaggle.com/CooperUnion/cardataset). Il contient $11902$ observations de $14$ features présentées dans le tableau ci-dessous. Il s'agit de pouvoir prédire le prix de voitures à partir de certaines de leur caractéristiques.


|Nom  |Description|Valeurs|
|-----|-----------|--------------------|
|Make  | marque | {BMW, FIAT, Mercedes-Benz, Chrysler, Nissan, Volvo,|
|   | |    Mazda, Mitsubishi, Ferrari, Alfa Romeo, Toyota,|
|   | |         McLaren, Maybach, Pontiac, Porsche, Saab, Audi, GMC,|
|   | |       Hyundai, Plymouth, Honda, Oldsmobile, Suzuki, Ford,|
|   | |       Cadillac, Kia, Bentley, Chevrolet, Dodge, Lamborghini,|
|   | |       Lincoln, Subaru, Volkswagen, Spyker, Buick, Acura,|
|   | |       Rolls-Royce, Maserati, Lexus, Aston Martin, Land Rover,|
|   | |       Lotus, Infiniti, Scion, Genesis, HUMMER, Tesla,|
|   | |       Bugatti}||
|Year| année de fabrication  |$\mathbb{N}$|
|Engine Fuel Type| type de carburant |  {0 : 'unleaded', 1 : 'flex-fuel', 2 : 'diesel',|
| |  | 3 : 'electric',4 : 'natural gas'} |
|Engine HP| puissance du moteur (hp) | $\mathbb{R}$|
|Engine Cylinders | nombre de cylindre |$\mathbb{N}$|
|Transmission Type | type de transmission |'MANUAL', 'AUTOMATIC', 'AUTOMATED_MANUAL', 'DIRECT_DRIVE',
 |   | |      'UNKNOWN'|
|Driven_Wheels| roues motrices |{rear wheel drive, all wheel drive, front wheel drive,four wheel drive}|
|Number of Doors| nombre de portes | $\mathbb{N}$|
|Vehicle Size| taille du véhicule | {Compact, Midsize, Large}|
|highway MPG| consommation sur route (en mpg)| $\mathbb{R}$|
|city mpg| consommation en ville (en mpg) |  $\mathbb{R}$|
|Popularity| popularité (via twitter) | $\mathbb{R}$|
|MSRP| prix (en \\$) | $\mathbb{R}$|

><u>Tâche 1</u>. Importez et visualisez les données se trouvant dans le fichier *data_cars.csv* fourni. Vous pouvez utiliser pour cela les méthodes suivantes, se trouvant dans la librairie <u>pandas</u> :
>- <span style="border:1px; border-style:solid">read_csv</span> 
>- <span style="border:1px; border-style:solid">head</span> 
>- <span style="border:1px; border-style:solid">describe</span>, en particulier, vous pouvez donner à cette fonction le paramètre suivant : *include = 'object'* ou *include = 'int64'* ou *include = 'float64'*, etc ... pour isoler un certain groupe de features.
>
>ainsi que les attributs :
>- <span style="border:1px; border-style:solid">dtypes</span> 
>- <span style="border:1px; border-style:solid">shape</span>
>
> Assurez-vous que le nombre d'observations et de variables correspond bien à ce qui vous a été annoncé en introduction.
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

><u>Tâche 2</u>. Repérez la feature qui est de type 'int64' mais qu'il serait plus logique de transformer en 'object' car c'est une variable catégorielle.
>
> Effectuez cette transformation à l'aide du code proposé ci-dessous.

In [9]:
#mettre au bon format
mapping_??? = {
        ?? : ??,
        ?? : ??,
        ?? : ??,
        ?? : ??
}
data[???]=data[???].replace(mapping_???)

><u>Tâche 3</u>. Quelles sont les données :
>- quantitatives ? 
>- catégorielles ? 
>- ordinales ? 
>
> Complétez les listes suivantes.

In [10]:
list_q=[]#variables quantitatives sans la variable à prédire 'MSPR' 
list_c=[]#variables catégorielles
list_o=[]#variables ordinales

><u>Tâche 4</u>. Labellisez la variable ordinale à l'aide du code ci-dessous :

In [12]:
#mettre au bon format
mapping_ordinal = {
        ??? : ???,
        ??? : ???,
        ???: ???
}
data[???]=data[???].replace(mapping_ordinal)

><u> Tâche 5</u>. Visualisez les gammes de prix des différentes marques de véhicule à l'aide de la fonction <span style="border:1px; border-style:solid">boxplot</span> de la librairie <u>seaborn</u>.

In [1]:
plt.figure(figsize=(16, 12))
sns.boxplot(x=???, y=???, data=???)

><u> Tâche 6</u>. Il apparaît que ce jeu de données renferme des marques de véhicules aux gammes de prix très hétérogènes. Visualisez le prix médian de chaque marque à l'aide de la fonction <span style="border:1px; border-style:solid">groupby</span> de la librairie <u>pandas</u>.

In [2]:
Make=data.groupby([???])[???].median()
plt.figure(figsize=(16, 12))
Make.plot(kind='bar', stacked=True)

><u> Tâche 7</u>. Il semble judicieux de traiter des groupes de marques différemment selon leur gamme de prix. A l'aide des fonctions <span style="border:1px; border-style:solid">groupby</span> et <span style="border:1px; border-style:solid">join</span> de la librairie <u>pandas</u>, gardez uniquement dans le jeu de données les marques de prix médian inférieur à $30000$\\$.

In [3]:
data=data.join(data.groupby([???])[???].median(), on='Make', rsuffix='_Median')
data=data[data['MSRP_Median']<???]
data=data.drop(['MSRP_Median'],axis=1)
data.reset_index(drop=True)

><u> Tâche 8</u>. Vérifiez qu'il n'il y pas d'outliers dans les valeurs des prix que nous avons gardé avec la fonction <span style="border:1px; border-style:solid">boxplot</span> de la librairie <u>seaborn</u> et enlevez-les le cas échéant. Combien de marques avons-nous gardées jusqu'à présent ?

In [4]:
sns.boxplot(x=???)

In [5]:
data=data[data['MSRP']<???]
data.reset_index(drop=True,inplace=True)

In [6]:
data['Make'].nunique()

On restreint encore le nombre catégories en sélectionnant uniquement les marques suivantes : 'Ford', 'Chevrolet', 'Chrysler','Pontiac', 'Subaru', 'Hyundai', 'Honda', 'Mazda', 'Nissan' et 'Suzuki'.

In [21]:
Makes=['Ford','Chevrolet','Chrysler','Pontiac','Subaru',
       'Hyundai','Honda','Mazda', 'Nissan','Suzuki']
data=data[data.Make.isin(Makes)]
data.reset_index(drop=True,inplace=True)

><u>Tâche 9</u>. Effectuez une analyse préliminaire des relations entre ces variables, qui pourra nous mettre sur la piste des transformations et créations de feature à réaliser. Pour cela :
>
>- Regardez quelles sont les corrélations entre les variables quantitatives à l'aide des fonctions <span style="border:1px; border-style:solid">corr</span> de <u>pandas</u> et <span style="border:1px; border-style:solid">heatmap</span> de la librairie <u>seaborn</u>. Même chose pour la variable ordinale.
>- Visualiser les nuages de points et leur tendance linéaire ou non entre les différentes variables quantitatives et la variable que l'on veut prédire, 'MSRP', en utilisant la fonction <span style="border:1px; border-style:solid">regplot</span> de la librairie <u>seaborn</u>. Même chose pour la variable ordinale.
>- Visualiser les boxplots des différentes variables catégorielles relativement à la variable 'MSPR' en utilisant la fonction <span style="border:1px; border-style:solid">boxplot</span> de la librairie <u>seaborn</u>.
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

In [7]:
corr = data[???].corr()
sns.heatmap(???, vmin=-1, vmax=1)

In [8]:
sns.regplot(x=???, y=???, data=???)

In [9]:
sns.boxplot(x=???, y=???, data=???)

'Highway MPG' et 'city mpg' étant fortement corrélées, on ne garde que 'highway MPG' dans le jeu de données. On écarte également 'Popularity' qui semble peu pertinent.

In [28]:
data=data.drop(['city mpg','Popularity'],axis=1)

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 2 Transformations essentielles

### 2-1 One-hot encoding

Nous allons maintenant transformer les variables catégorielles en utilisant l'encodage one-hot vu en cours. Pour cela nous utiliserons la fonction <span style="border:1px; border-style:solid">OneHotEncoder</span> se trouvant dans le module <u>preprocessing</u> de la librairie <u>sklearn</u>. Installez-la si nécessaire et importez-la :

In [30]:
from sklearn.preprocessing import OneHotEncoder

><u>Tâche 10</u>. Le code ci-dessous détaille pas-à-pas comment effectuer l'encodage one-hot. Complétez-le. (Pensez à utiliser les listes que nous avons définies). Encodez aussi les variables 'Number of Doors', 'Vehicle Size' et 'Engine Cylinders' en one-hot. Pouvez-vous expliquez pourquoi ?

In [31]:
list_one_hot=???
#On crée l'objet 'encoder_OH'
encoder_OH = OneHotEncoder(handle_unknown='error',sparse=False)
#On ajuste nos données à l'objet 'encoder_OH'
fit_OH=encoder_OH.fit(data[???])
#On transforme nos données catégorielles grâce à l'objet 'encoder_OH'
fit_data_OH=fit_OH.transform(data[???])
#On récupère les noms pour nos nouvelles colonnes
column_name_OH =fit_OH.get_feature_names(list_one_hot)
#On met nos données transformées sous forme de Dataframe
cols_OH = pd.DataFrame(???,columns=???)
#on réinitialise l'index
cols_OH.reset_index(drop=True, inplace=True)

><u>Tâche 11</u>. Nous allons vérifier que le code précédent est correct, c'est-à-dire qu'il effectue bien ce qu'on attend de lui. 
>- Combien de variables avions-nous avant transformation ? Avec combien de catégories chacunes ? Combien de variables 'one-hot' avons nous après transformations ? Cela est-il cohérent ?
>- Visualisez le nom des colonnes de nos nouvelles variables en utilisant la commande <span style="border:1px; border-style:solid">list(cols_OH.columns)</span>. Est-ce que ces noms sont cohérents et utiles ?
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

>Remarque : en cours, nous avons utilisé la fonction <span style="border:1px; border-style:solid">get_dummies</span> de <u>pandas</u> pour effectuer l'encodage one-hot. Pour une comparaison entre get_dummies et OneHotEncoder, voir la fin.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 2-2 Mise à l'échelle

Nous allons maintenant mettre à l'échelle les variables quantitatives restantes en utilisant la standardisation vue en cours. Pour cela nous utiliserons la fonction <span style="border:1px; border-style:solid">StandardScaler</span> se trouvant dans le module <u>preprocessing</u> de la librairie <u>sklearn</u>. Installez-la si nécessaire et importez-la :

In [37]:
from sklearn.preprocessing import StandardScaler

><u>Tâche 12</u>. Le code ci-dessous détaille pas-à-pas comment effectuer la standardisation. Complétez-le.

In [38]:
list_std=???
#On crée l'objet 'scale'
scale = StandardScaler()
#On ajuste nos données à l'objet 'scale'
fit_scale_OH=scale.fit_transform(data[???])
#On met sous forme de Dataframe
cols_scaled = pd.DataFrame(???,columns=list_std)
cols_scaled.reset_index(drop=True, inplace=True)

><u>Tâche 13</u>. Nous allons vérifier que le code précédent est correct, c'est-à-dire qu'il effectue bien ce qu'on attend de lui. 
>- Combien de variables avions-nous avant transformation ? Combien de variables avons-nous après transformation ?
>- Visualisez les nouvelles variables en utilisant la commande <span style="border:1px; border-style:solid">describe</span>. Est-ce que cela est cohérent ?
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

><u>Tâche 14</u>. Concaténez les deux parties (variables quantitatives standardisées et variables encodées one-hot en un seul Dataframe *X_init_OH*.

In [40]:
X_init_OH = pd.concat([???, ???], axis=1)

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 2-3 Résultat de référence

Nous souhaitons vérifier que ces transformations sont bien utiles et que l'augmentation de la dimension qui en découle n'est pas délétère.

Pour cela nous allons utiliser les fonctions <span style="border:1px; border-style:solid">score_model_1</span> et <span style="border:1px; border-style:solid">score_model_2</span> suivantes. Il n'est pas nécessaire de comprendre exactement de ce que font ces fonctions, vous pouvez vous en servir comme d'une boîte noire qui vous indique à quel point vos données sont bien préparées : plus le score **'MSE' est petit**, **meilleur** est votre jeu de données et plus le score **'R2' est proche de $1$** (normalement $0\leq R^2\leq 1$) **meilleur** est votre jeu de données .

Nous utiliserons aussi les méthodes ci-dessous issues de <u>sklearn</u>. Installez-les si nécessaire et importez-les :

In [43]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

In [44]:
def score_model_1(X_train,y_train,X_test,y_test):
    #on crée l'objet modèle linéaire
    regr = LinearRegression()

    #on ajuste le modèle sur les données d'entrainement et on enregistre et affiche le temps nécessaire pour cela
    start = time.time()
    regr.fit(X_train, y_train)
    end = time.time()
    print('Temps d execution du modèle 1 : ' "{:e}".format(end - start))

    #On fait des prédictions sur les données de l'enseble de test
    prix_y_pred = regr.predict(X_test)
    
    #On affiche le score du modèle
    print('Score MSE sur le modele 1 : ')
    print("{:e}".format(mean_squared_error(y_test, prix_y_pred)))
    print('Score R2 sur le modele 1 : ')
    print(r2_score(y_test, prix_y_pred))

In [45]:
def score_model_2(X_train,y_train,X_test,y_test):
    #on crée l'objet modèle linéaire
    regr = SGDRegressor(max_iter=500,penalty=None,eta0=0.01,random_state=42)
    
    #on ajuste le modèle sur les données d'entrainement et on enregistre et affiche le temps nécessaire pour cela
    start = time.time()
    regr.fit(X_train, y_train)
    end = time.time()
    print('Temps d execution du modèle 2 : ' "{:e}".format(end - start))

    #On fait des prédictions sur les données de l'enseble de test
    prix_y_pred = regr.predict(X_test)
    
    #On affiche le score du modèle
    print('Score MSE sur le modele 2 : ')
    print("{:e}".format(mean_squared_error(y_test, prix_y_pred)))
    print('Score R2 sur le modele 2 : ')
    print(r2_score(y_test, prix_y_pred))

>Expliquons tout de même brièvement en quoi consistent ces fonctions (la lecture de ce paragraphe n'est pas nécessaire à la poursuite du TP et vous pouvez le sauter sans remord). Les deux fonctions prennent en entrée des données d'apprentissage, les prédicteurs (ou variables explicatives) et les réponses (ou variables à expliquer) correspondantes. Avec ces données, on entraîne un modèle, c'est-à-dire qu'on cherche les paramètres optimaux de ce modèle. La fonction *score_model_1* fait une régression linéaire (on étudiera cela bientôt) avec les paramètres par défaut de l'algorithme donné par sklearn et la fonction *score_model_2* fait une régression linéaire plus élaborée. 
>
>Une fois les paramètres trouvés, on veut voir si ce modèle est bon. On va donc lui demander de prédire des prix sur des données qu'il n'a jamais vu avant. Si le modèle est bon, il va trouver à partir de de l'ensemble de test des valeurs plutôt proches des vrais prix. Pour savoir à quel point on est porche de du vrai prix, on calcule [l'erreur quadratique moyenne](https://en.wikipedia.org/wiki/Mean_squared_error#Predictor) entre les prédictions et les vrais prix : plus elle est petite, meilleur est le modèle. Nous donnons aussi le [coefficient de détermination](https://fr.wikipedia.org/wiki/Coefficient_de_d%C3%A9termination) : plus il est proche de un est meilleur est le modèle.

Enfin, nous avons besoin de séparer *X_init_OH* en deux parties (l'ensemble d'apprentissage et l'ensemble de test).

In [46]:
X_train, X_test, y_train, y_test  = train_test_split(X_init_OH,data['MSRP'], test_size=0.2,random_state=42)
idx=X_train.index.values
idxT=X_test.index.values

><u>Tâche 15</u>. Tout est prêt pour mesurer la qualité de notre jeu de données. Exécuter le code suivant et notez les scores et les temps d'exécution obtenus, cela nous servira à faire des comparaisons plus tard. 

In [2]:
X_train=X_init_OH.iloc[idx]
y_train=data['MSRP'].iloc[idx]
X_test=X_init_OH.iloc[idxT]
y_test=data['MSRP'].iloc[idxT]
score_model_1(X_train,y_train,X_test,y_test)
print('\n')
score_model_2(X_train,y_train.values.ravel(),X_test,y_test)

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 2-4 One-hot encoding vs label encoding

Le but de ce paragraphe est de vous convaincre que l'encodage one-hot est bien utile. A cette fin, nous allons donc faire **<u>ce qu'il ne faut jamais faire en pratique</u>**, à savoir labelliser les variables catégorielles. Nous utilisons pour cela la fonction <span style="border:1px; border-style:solid">OrdinalEncoder</span> de <u>sklearn</u> (et d'ailleurs, comme son nom l'indique, cette fonction doit être réservée aux variables ordinales). Nous garderons <u>les variables quantitatives mises à l'échelle</u> dans cette partie.

In [48]:
from sklearn.preprocessing import OrdinalEncoder

><u>Tâche 16</u>. Complétez le code suivant afin de labelliser les variables catégorielles. Vérifier ensuite, comme dans les tâches précédentes, que le code que vous avez écrit fait bien ce qu'on attend de lui.

In [49]:
#On crée l'objet 'encoder_O'
encoder_O = OrdinalEncoder()
#On ajuste nos données à l'objet 'encoder_O'
fit_O=???
#On transforme nos données catégorielles grâce à l'objet 'encoder_OH'
fit_data_O=???
#On met nos données transformées sous forme de Dataframe
cols_O = ???
#on réinitialise l'index
cols_O.reset_index(drop=True, inplace=True)

><u>Tâche 17</u>. Concaténez les deux parties (variables quantitatives standardisées et variables catégorielles labellisées) en un seul Dataframe *X_init_O*.

><u>Tâche 18</u>. Appliquer les fonctions *score_model_1* et *score_model_2* pour connaître les scores et les temps d'éxecution des modèles 1 et 2 en utilisant ces données.  
>
>Retrouve-t-on ce qu'on a vu en cours, à savoir, que le label encoding est une très mauvaise idée pour des données catégorielles ? Le temps d'exécution est-il impacté par l'encodage one-hot sur ce jeu de données ?
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

In [1]:
X_train=???
y_train=???
X_test=???
y_test=???
score_model_1(X_train,y_train,X_test,y_test)
print('\n')
score_model_2(X_train,y_train,X_test,y_test)

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 2-5 Oubli de mise à l'échelle

Le but de ce paragraphe est de vous sensibiliser à la mise à l'échelle. A cette fin, nous allons donc faire l'inverse de la partie précédente : garder les variables quantitatives brutes (non mises à l'échelle) et utiliser l'encodage one-hot de la section 2-1.

En théorie, on s'attend à ce que le modèle 1 ne soit pas impacté (la régression linéaire ne repose pas sur des distances) alors que le modèle 2 peut l'être (il effectue une descente de gradient, qui est sensible aux ordres de magnitude des features).

><u>Tâche 19</u>. Créer un nouveau Dataframe *X_non_scaled* contenant les variables quantitatives non mises à l'échelle et les variables catégorielles one-hot.
>
>Appliquer les fonctions *score_model_1* et *score_model_2* pour connaître les scores et les temps d'éxecution des modèles 1 et 2 en utilisant ces données.  
>
>Retrouve-t-on ce à quoi on s'attend en théorie ? Le temps d'exécution est-il impacté ?
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 2-6 Sur les données non standardisées et mal encodées

Ce dernier paragraphe de la section a pour objet de bien enfoncer le clou : que se passe-t-il si on oublie les deux points essentiels de ce cours : l'encodage one-hot et la mise à l'échelle ?

><u>Tâche 20</u> (facultative). Créer un nouveau Dataframe *X_none* contenant les variables quantitatives non mises à l'échelle et les variables catégorielles labellisées.
>
>Appliquer les fonctions *score_model_1* et *score_model_2* pour connaître les scores et les temps d'exécution des modèles 1 et 2 en utilisant ces données.  
>
>Retrouve-t-on ce à quoi on s'attend en théorie ?
>
> Pensez à utiliser autant de cellules de code que nécessaire pour avoir un document agréable à lire et bien organisé.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 3 Création de features

### 3-1 Le cas de la variable 'Year'

><u>Tâche 21</u>. Quand on cherche à acheter une voiture, on va plutôt considérer l'âge de la voiture. Créez la variable 'Age' à partir de la variable 'Year' et substituez là à celle-ci puis tester cette modification avec *score_model_1* et *score_model_2*.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 3-2 Le cas de la variable 'highway MPG'

><u>Tâche 22</u>. L'unité mpg, miles per gallon, est utilisé aux Etats-Unis. En France, on parle plutôt de litre au cent (km). Ajoutez au jeu de données une nouvelle variable 'highway Lkm' qui est donc la consommation en litre aux $100$ kilomètres. Est-ce que cela améliore notre jeu de données (n'oubliez pas de standardiser la nouvelle variable !).

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 3-3 Terme d'interaction

><u>Tâche 23</u>. Créer une nouvelle variable 'Age_MPG' qui est égale à l'âge du véhicule multiplié par sa consommation de carburant en conduite sur route 'Highway MPG'. Cela améliore-t-il notre jeu de données (n'oubliez pas de standardiser la nouvelle variable !) ?

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

### 3-4 Encore des transformations ...

><u>Tâche 24</u>. Trouvez quelles feature $f(x)$ ajouter au jeu de données pour $f\in \{\log(\cdot), \sqrt{\cdot}, \sqrt[3]{\cdot}\}$ et $x\in \{\text{'Engine HP', 'highway MPG'}\}$. Pour cela vous pouvez :
>- Tracer les nuages de points $(f(x),\text{'MSRP'})$
>- Regarder les coefficients de corrélations de $f(x)$ avec $\text{'MSRP'}$.
> Une fois votre choix fait, testez votre jeu de données comme précédemment.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 4 Et les variables catégorielles ?

### 4-1 Sont-elles toutes déterminantes ?

><u>Tâche 25</u>. Si certaines features sont très corrélées avec d'autres, ou bien si elles sont complètement sans effet sur le prédicteur, cela peut diminuer les résultats d'un modèle. Les résultats sont-ils meilleurs avec ou sans les variables 'Vehicle size' et 'Number of Doors' ? Pouvait-on le prévoir ?

><u>Commentaire.</u> Il existe des méthodes rigoureuses pour choisir les variables les plus pertinentes. Cela s'appelle la *sélection de features* et fait l'objet d'un cours en particulier pour ceux d'entre vous qui continueront un cursus data science.

><u>Tâche 26</u>. Tracez le nuage de points de la feature 'MSPR' en fonction de 'highway MPG' avec les labels des classes de 'Transmission Type'. Commentez.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 5 OneHotEncoder vs get_dummies

<span style="border:1px; border-style:solid">OneHotEncoder</span> se trouve dans la librairie de <u>sklearn</u> alors que <span style="border:1px; border-style:solid">get_dummies</span> est une fonction de <u>pandas</u>. Ils ont chacun des avantages et des inconvénients, cependant *à l'heure actuelle* (sous réserve de mise à jour de ces librairies), je vous conseillerais d'utiliser OneHotEncoder.

- get_dummies :    
    - (+) crée directement un Dataframe à partir du Dataframe à encoder.
    - (-) ne prend en entrée que des chaînes de caractère.
    - permet de faire un encodage dummy avec l'option 'drop_first=True' (mais pas le choix de quelle variable on enlève).

- OneHotEncoder :
    - (-) retourne un objet de type 2-d array (ou une matrice), et non pas un Dataframe.
    - (+) prend en entrée des chaînes de caractère et des entiers.
    - permet de faire un encodage dummy avec l'option 'drop' (et de choisir quelle entrée on enlève).
    - **crée une fonction persistente**, de sorte que si vous voulez encoder de nouvelles données, l'utilisation de cette fonction vous assure d'avoir des résultats consistants. Il y a deux options : handle_unknown="ignore" ou handle_unknown="error". Dans le cas handle_unknown="error", si vous donnez à l'encodeur un jeu de données avec une nouvelle catégorie, qui n'était pas dans le premier jeu de données tranformé, alors cela retourne une erreur.
    
Par exemple : si votre premier jeu de données comporte une variables avec les catégories "rouge", "vert" et "jaune" et que votre deuxième jeu de données contient en plus la catégorie "bleue", alors :

- get_dummies va créer la colonne 'couleur_bleue' dans l'encodage one-hot, ce qui entraînera des problèmes plus loin dans votre code si vos modèles ont été entraînés sans catégorie 'bleue'.
- OneHotEncoder vous signalera une erreur (handle_unknown="error") ou bien ignorera cette nouvelle catégories (handle_unknown="ignore") (c'est-à-dire mettre des $0$ dans les colonnes 'couleur_rouge', 'couleur_vert' et 'couleur_jaune'.
    
En résumé, get_dummies est facile à manipuler et peut être privilégié pour de l'exploration, avec des petites quantités de données, et OneHotEncoder est préférable dans des projets plus complexes ou l'on veut contrôler avec soin toutes les variables, voir [ici](https://stackoverflow.com/questions/36631163/pandas-get-dummies-vs-sklearns-onehotencoder-what-are-the-pros-and-cons).     

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>